# Analysis of my four settings

In [63]:
# modules
import xmltodict
import pandas as pd

# reading in data
#loading the file
with open("BLOSUM50_50.xml","r") as read_file:
    content = read_file.read()
    run1 = xmltodict.parse(content)

#loading the file
with open("BLOSUM62_50.xml","r") as read_file:
    content = read_file.read()
    run2 = xmltodict.parse(content)

#loading the file
with open("BLOSUM50_50_noGapPenalty.xml","r") as read_file:
    content = read_file.read()
    run8 = xmltodict.parse(content)

#loading the file
with open("BLOSUM50_50_maxGapPenalty.xml","r") as read_file:
    content = read_file.read()
    run9 = xmltodict.parse(content)

rundict = {}

rundict["run1"] = run1
rundict["run2"] = run2
rundict["run8"] = run8
rundict["run9"] = run9

## comparison BLOSUM50 BLOSUM62

In [169]:
# making a big dataframe with all sort of useful information

anal1 = pd.DataFrame(columns=["matrix","match_name","match_ID","Evalue","Identity","optimalScore"])
for key,value in rundict.items():
    for i in range(50):
        anal1.loc[str(key)+"_"+str(i),"Identity"]= value["EBIApplicationResult"]["SequenceSimilaritySearchResult"]["hits"]["hit"][i]["alignments"]["alignment"]["identity"]
        anal1.loc[str(key)+"_"+str(i),"optimalScore"]= value["EBIApplicationResult"]["SequenceSimilaritySearchResult"]["hits"]["hit"][i]["alignments"]["alignment"]["opt"]
        anal1.loc[str(key)+"_"+str(i),"overlap"]= value["EBIApplicationResult"]["SequenceSimilaritySearchResult"]["hits"]["hit"][i]["alignments"]["alignment"]["overlap"]
        anal1.loc[str(key)+"_"+str(i),"Evalue"]= value["EBIApplicationResult"]["SequenceSimilaritySearchResult"]["hits"]["hit"][i]["alignments"]["alignment"]["expectation"]
        anal1.loc[str(key)+"_"+str(i),"match_name"]= value["EBIApplicationResult"]["SequenceSimilaritySearchResult"]["hits"]["hit"][i]["@id"]
        anal1.loc[str(key)+"_"+str(i),"match_ID"]= value["EBIApplicationResult"]["SequenceSimilaritySearchResult"]["hits"]["hit"][i]["@ac"]
        anal1.loc[str(key)+"_"+str(i),"matrix"]= value["EBIApplicationResult"]["Header"]["parameters"]["matrix"]
        anal1.loc[str(key)+"_"+str(i),"run"]= key
    
# only looking at run1 and 2  
anal1 = anal1[(anal1["run"]=="run1") | (anal1["run"]=="run2")]
print(anal1)

        matrix   match_name  match_ID    Evalue Identity optimalScore overlap  \
run1_0    BL50  WNT5A_HUMAN    P41221  2.8e-174    100.0         2662     380   
run1_1    BL50  WNT5A_RABIT    Q27Q52  4.1e-173     98.9         2645     380   
run1_2    BL50  WNT5A_MOUSE    P22725  2.7e-172     98.7         2633     380   
run1_3    BL50    WNT5A_RAT    Q9QXQ7    7e-172     98.4         2627     380   
run1_4    BL50  WNT5A_HUMAN  P41221-2  6.1e-168    100.0         2569     365   
...        ...          ...       ...       ...      ...          ...     ...   
run2_45   BL62   WNT4_MOUSE    P22724   2.3e-95     50.0          889     328   
run2_46   BL62   WNT2_MUSPF    Q07E18   2.4e-95     52.7          889     317   
run2_47   BL62   WNT2_DASNO    Q07E44   3.1e-95     52.7          888     317   
run2_48   BL62   WNT2_CHLAE    Q2IBB0   4.1e-95     52.4          887     317   
run2_49   BL62   WNT2_MUNMU    Q09YJ6   4.1e-95     52.7          887     317   

          run  
run1_0   ru

In [179]:
# I want a dataframe where I can look at the same values for identiy, evalue etc for each matrix next to each other
# so I can compare them better, so I reorder my data a bit:
# the new dataframe should have one row for each match_ID found in run1 and run2:

# I access all match IDs and turn them into a set to remove duplicates, 
# I then turn the whole thing into a list so I can use them as row names
direct_comp = pd.DataFrame(index = list(set(anal1["match_ID"])))
direct_comp

""
A0M8T2
P21552
P22725-2
P56705
Q93097-2
Q09YJ6
Q9H1J7
Q5NVK2
Q06443
Q27Q52


In [ ]:
# then I add one by one for each match ID the values for 
# - Identity in 2 colums (for the two matrices)
# - Evalue in 2 colums
# - optimal score in 2 columns
# - overlap in 2 columns

In [180]:
for match in direct_comp.index:
    if any(match == elem for elem in anal1[anal1["run"]=="run1"]["match_ID"]) :
        direct_comp.loc[match,"Identity_BL50"] = float(list(set(anal1[(anal1["run"]=="run1") & (anal1["match_ID"] == match)]["Identity"]))[0])
    if any(match == elem for elem in anal1[anal1["run"]=="run2"]["match_ID"]) :
        direct_comp.loc[match,"Identity_BL62"] = float(list(set(anal1[(anal1["run"]=="run2") & (anal1["match_ID"] == match)]["Identity"]))[0])
    

In [181]:
for match in direct_comp.index:
    if any(match == elem for elem in anal1[anal1["run"]=="run1"]["match_ID"]) :
        direct_comp.loc[match,"Evalue_BL50"] = float(list(set(anal1[(anal1["run"]=="run1") & (anal1["match_ID"] == match)]["Evalue"]))[0])
    if any(match == elem for elem in anal1[anal1["run"]=="run2"]["match_ID"]) :
        direct_comp.loc[match,"Evalue_BL62"] = float(list(set(anal1[(anal1["run"]=="run2") & (anal1["match_ID"] == match)]["Evalue"]))[0])
    

In [182]:
for match in direct_comp.index:
    if any(match == elem for elem in anal1[anal1["run"]=="run1"]["match_ID"]) :
        direct_comp.loc[match,"optimalScore_BL50"] = float(list(set(anal1[(anal1["run"]=="run1") & (anal1["match_ID"] == match)]["optimalScore"]))[0])
    if any(match == elem for elem in anal1[anal1["run"]=="run2"]["match_ID"]) :
        direct_comp.loc[match,"optimalScore_BL62"] = float(list(set(anal1[(anal1["run"]=="run2") & (anal1["match_ID"] == match)]["optimalScore"]))[0])
    

In [183]:
for match in direct_comp.index:
    if any(match == elem for elem in anal1[anal1["run"]=="run1"]["match_ID"]) :
        direct_comp.loc[match,"overlap_BL50"] = float(list(set(anal1[(anal1["run"]=="run1") & (anal1["match_ID"] == match)]["overlap"]))[0])
    if any(match == elem for elem in anal1[anal1["run"]=="run2"]["match_ID"]) :
        direct_comp.loc[match,"overlap_BL62"] = float(list(set(anal1[(anal1["run"]=="run2") & (anal1["match_ID"] == match)]["overlap"]))[0])
    

In [187]:
# since when using set I got a random order of my match IDs I sort the whole thing by E-value of the 
# default matrix BL50, followed by E-value of tmatrix BL62
direct_comp.sort_values(by = ["Evalue_BL50","Evalue_BL62"],inplace=True)


> Now I can easily compare the two settings

- BL62 has generally lower E-values
- identity is generally the same, if there are differences BL62 has the higher identity, except for two cases (Q93097 and Q93097-2)
- the score is always higher for BLOSUM50
- BL50 has a higher alignment overlap if there are differences

> I think they are both good, so it's ok to stick to default

In [188]:
direct_comp

,Identity_BL50,Identity_BL62,Evalue_BL50,Evalue_BL62,optimalScore_BL50,optimalScore_BL62,overlap_BL50,overlap_BL62
P41221,100.0,100.0,2.800000e-174,0.000000e+00,2662.0,2061.0,380.0,380.0
Q27Q52,98.9,98.9,4.100000e-173,0.000000e+00,2645.0,2048.0,380.0,380.0
P22725,98.7,98.7,2.700000e-172,0.000000e+00,2633.0,2038.0,380.0,380.0
Q9QXQ7,98.4,98.4,7.000000e-172,0.000000e+00,2627.0,2033.0,380.0,380.0
P41221-2,100.0,100.0,6.100000e-168,0.000000e+00,2569.0,1989.0,365.0,365.0
P22725-2,99.4,99.4,3.200000e-165,0.000000e+00,2529.0,1958.0,360.0,360.0
P31286,85.3,87.3,3.800000e-152,7.800000e-201,2338.0,1795.0,381.0,370.0
O13267,89.2,89.2,3.400000e-150,2.400000e-199,2309.0,1782.0,360.0,360.0
Q06442,87.5,87.5,3.000000e-147,2.200000e-195,2266.0,1748.0,360.0,360.0
Q92050,81.7,81.7,7.500000e-139,2.900000e-183,2143.0,1644.0,360.0,360.0


In [193]:
# some calculations

calc = pd.DataFrame(index = direct_comp.index)
calc["Ident_50/60"] = direct_comp["Identity_BL50"]/direct_comp["Identity_BL62"]
calc["Eval_50/60"] = direct_comp["Evalue_BL50"]/direct_comp["Evalue_BL62"]
calc["Score_50/60"] = direct_comp["optimalScore_BL50"]/direct_comp["optimalScore_BL62"]
calc["overlap_50/60"] = direct_comp["overlap_BL50"]/direct_comp["overlap_BL62"]

calc

,Ident_50/60,Eval_50/60,Score_50/60,overlap_50/60
P41221,1.000000,inf,1.291606,1.000000
Q27Q52,1.000000,inf,1.291504,1.000000
P22725,1.000000,inf,1.291953,1.000000
Q9QXQ7,1.000000,inf,1.292179,1.000000
P41221-2,1.000000,inf,1.291604,1.000000
P22725-2,1.000000,inf,1.291624,1.000000
P31286,0.977090,4.871795e+48,1.302507,1.029730
O13267,1.000000,1.416667e+49,1.295735,1.000000
Q06442,1.000000,1.363636e+48,1.296339,1.000000
Q92050,1.000000,2.586207e+44,1.303528,1.000000
